In [ ]:
!pip uninstall nltk
!pip install nltk==3.5
!pip install rake-nltk

Found existing installation: nltk 3.7
Uninstalling nltk-3.7:
  Would remove:
    /usr/local/bin/nltk
    /usr/local/lib/python3.8/dist-packages/nltk-3.7.dist-info/*
    /usr/local/lib/python3.8/dist-packages/nltk/*
Proceed (Y/n)? y
  Successfully uninstalled nltk-3.7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434691 sha256=34cf23606b655c13102ca379f1849db867dbab9c62a82a92107b1285a2327f32
  Stored in directory: /root/.cache/pip/wheels/ff/d5/7b/f1fb4e1e1603b2f01c2424dd60fbcc50c12ef918bafc44b155
Successfully built nltk
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.7 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installati

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.cluster.util import cosine_distance
from sklearn.decomposition import LatentDirichletAllocation
import scipy.sparse
from rake_nltk import Rake
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.probability import FreqDist
import re
import collections
nltk.download('punkt');
nltk.download('stopwords');
nltk.download('vader_lexicon');

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
df = pd.read_csv('transcripciones_para_modelo.csv')

In [ ]:
compromiso = df[df['AGENDAMIENTO_ESTADO'].str.contains('COMPROMISO') == True]


In [ ]:
compromiso.drop(columns='level_0', inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
compromiso.reset_index(inplace=True)

In [ ]:
compromiso.columns

Index(['Unnamed: 0', 'level_0', 'index', 'Unnamed: 0.1', 'ID_MOPC', 'IDMOROSO',
       'IDCLIENTE', 'OPERADOR', 'CAMPAÑA', 'IDGRABACION', 'TIPO_DISCADOR',
       'FECHA', 'ID', 'IDLLAMADA', 'ESTADO', 'SUBESTADO', 'F_INICIO',
       'ANI_TELEFONO', 'GRABACION_DEP', 'TIPO', 'CRM', 'BASE', 'LOG',
       'DIRECCION', 'IDOPERADOR', 'AGENDAMIENTO_ESTADO', 'anos', 'mes',
       'transcripcion', 'transcripcion_limpia', 'Contestador'],
      dtype='object')

In [ ]:
compromiso['frases'] = None

<ipython-input-8-7853df584320>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compromiso['frases'] = None


In [ ]:
for n in range(len(compromiso)):
  compromiso['frases'][n] = re.split(r'\s{2,}', compromiso.transcripcion_limpia[n])


<ipython-input-9-8a683c20db57>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compromiso['frases'][n] = re.split(r'\s{2,}', compromiso.transcripcion_limpia[n])


In [ ]:
type(compromiso['frases'][3])

list

In [ ]:
len(compromiso)

468

In [ ]:
# import pandas as pd
# import re
# import random

# def seleccionar_frases(df):
#     # Crea una lista vacía para almacenar las frases seleccionadas
#     frases_seleccionadas = []

#     # Accede a la columna de transcripciones
#     transcripciones = df['transcripcion_limpia']

#     # Recorre cada transcripción
#     for transcripcion in transcripciones:
#         # Busca un patrón en la transcripción
#         match = re.search(r'monto', transcripcion)
#         if match:
#             # Divide la transcripción en frases
#             phrases = re.split(r'\s{2,}', transcripcion)
#             # Selecciona una frase al azar
#             selected_phrase = random.choice(phrases)
#             # Añade la frase seleccionada a la lista
#             frases_seleccionadas.append(selected_phrase)

#     # Devuelve la lista de frases seleccionadas
#     return frases_seleccionadas



In [ ]:
# # import pandas as pd
# from collections import Counter
# from nltk.tokenize import word_tokenize

# # # Carga el dataframe
# # df = pd.read_csv('transcripciones.csv')

# # Itera sobre cada fila del dataframe
# for index, row in compromiso.iterrows():
#   # Extrae la lista de frases de la fila
#   frases = row['frases']
  
#   # Tokeniza cada frase y crea una lista de todas las palabras
#   palabras = []
#   for frase in frases:
#     palabras += word_tokenize(frase)
  
#   # Calcula la frecuencia de cada palabra
#   frecuencia = Counter(palabras)
  
#   # Selecciona las palabras más frecuentes como posibles temas en común
#   temas = [palabra for palabra, count in frecuencia.most_common(5)]
  
#   # Extrae las frases que contengan las palabras seleccionadas
#   frases_tema = []
#   for frase in frases:
#     if any(tema in frase for tema in temas):
#       frases_tema.append(frase)
      
#   # Imprime las frases relacionadas con el tema
#   print(f'Temas en común para la fila {index}: {temas}')
#   print(f'Frases relacionadas: {frases_tema}')


In [ ]:
temas = ['pago',
 'deuda',
 'cuotas',
 'pesos',
 'whatsapp',
 'pagar',
 'hoy',
 'telefono',
 'cuenta',
 'momento',
 'saldo',
 'llamar',
 'nombre',
 'habla',
 'tarjeta',
 'hablo',
]
# Itera sobre cada tema
for tema in temas:
  # Lista de frases por tema
  frases_tema = []
  
  # Itera sobre cada fila del dataframe
  for index, row in compromiso.iterrows():
    # Extrae la lista de frases de la fila
    frases = row['frases']
    
    # Itera sobre cada frase
    for frase in frases:
      # Comprueba si la frase contiene el tema actual
      if tema in frase:
        # Añade la frase a la lista de frases por tema
        frases_tema.append(frase)
        break
        
  # Imprime la lista de frases por tema
  print(f'Frases relacionadas con el tema {tema}: {frases_tema}')

Frases relacionadas con el tema pago: ['el pago tiene que estar antes del mediodia si', 'si senor como verifico que usted ya pago la deuda entonces estaria quedando con el', 'que yo siempre tenia voluntad de pago', 'ah bueno si quieren enviarlo de vuelta para tenerlo mas fresco pero apenas me depositen voy y les pago', 'lo recuerdo y siempre pago un poquito de esto despues', 'eso te queria decir porque ella pago', 'para cuando podriamos agendar otra vez su pago', 'mientras yo espero el pago la acreditacion del pago de ayer si', 'y si usted esta autorizada para cerrar un acuerdo de pago', 'o no se o cupones para pagar en un rojo y pago o la cuenta para hacer la transferencia', 'la primera es obviamente por pago al contado', 'no me dieron los datos del estudio para que abone por un pago facil', 'por el pago del banco frances del dia de hoy', 'o mercado pago tambien puede ir directamente al banco santander rio nosotros le mandamos', 'realizado el plan de pago', 'el primer pago tiene que e

In [ ]:
filas = []

# Itera sobre cada fila del dataframe
for index, row in compromiso.iterrows():
  # Extrae la lista de frases de la fila
  frases = row['frases']
  
  # Crea un diccionario vacío para almacenar las frases por tema para la fila actual
  frases_por_tema = {}
  
  # Itera sobre cada tema
  for tema in temas:
    # Itera sobre cada frase
    for frase in frases:
      # Comprueba si la frase contiene el tema actual
      if tema in frase:
        # Añade la frase al diccionario si no existe aún una lista para el tema
        if tema not in frases_por_tema:
          frases_por_tema[tema] = []
        # Añade la frase a la lista del diccionario correspondiente al tema
        frases_por_tema[tema].append(frase)
  
  # Añade el diccionario a la lista de diccionarios
  filas.append(frases_por_tema)


In [ ]:
prueba = pd.DataFrame.from_records(filas)

In [ ]:
prueba

,pago,deuda,pesos,whatsapp,pagar,cuenta,llamar,cuotas,hoy,telefono,momento,nombre,habla,tarjeta,saldo,hablo
0,[el pago tiene que estar antes del mediodia si],[bien la deuda que presenta el dia de la fecha...,[pesos],[yo le voy a enviar tiene whatsapp en este num...,[perfecto pero lo puedo pagar el martes que vi...,[ahorita la voy a agregar al whatsapp y le voy...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[si senor como verifico que usted ya pago la d...,[si senor como verifico que usted ya pago la d...,[pesos si tengo que hacerle nuevamente un plan...,NaN,NaN,[bueno senor para que tengan cuenta entonces e...,[y bueno me tenes que llamar a fin de mes],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[que yo siempre tenia voluntad de pago],[que se enteraron de que tenia la deuda que te...,"[pesos, pesos]",[nosotros tenemos el contacto por whatsapp],[si si era la unica tarjeta no por falta de pa...,[toman la decision de dar de baja la cuenta po...,[volvimos a llamar porque sabemos],"[las cuotas, esta bonificacion y las cuotas]","[hoy desde poder informar, ahi esta mi situaci...",[el telefono por ejemplo],[esta sin trabajar usted esta sin trabajar en ...,[la tarjeta de la hbc estaba su nombre],[alejandra voy a dejarme de hablar del estudio...,"[nosotros teniamos una tarjeta del hbc, que me...",NaN,NaN
3,[ah bueno si quieren enviarlo de vuelta para t...,NaN,NaN,[igual tengo tu whatsapp asi que yo apenas dep...,NaN,NaN,NaN,NaN,[en estos dias tiene que ser lo te digo porque...,NaN,NaN,[porque vio que estamos cobrando con ese que a...,[si con el habla],NaN,NaN,NaN
4,NaN,[si necesitaria saber que me confirme hoy si v...,NaN,NaN,[si necesitaria saber que me confirme hoy si v...,NaN,[quiere llamarme mas tardecito yo me comunico ...,NaN,[si necesitaria saber que me confirme hoy si v...,NaN,[si yo en este momento no puedo hablar],NaN,[si yo en este momento no puedo hablar],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,[despues bueno veremos de aca a fin de mes y p...,[pedro lo contacto por su deuda con el banco s...,"[pesos, pesos cada uno lo ayudan a poder cance...",NaN,"[tendria que pagar una cuota y el otro, que vi...",NaN,NaN,NaN,[pesos hoy en dia no es nada es una salida con...,NaN,NaN,NaN,[queria hablar con usted para saber si va a re...,NaN,NaN,NaN
464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
compromiso.transcripcion_limpia[0]

'si maria me escucha este corton     si le escucho le escucho     bien la deuda que presenta el dia de la fecha son   pesos     y solo que le puedo ofrecer para que cancele con el descuento de este mes de abril     que vence el martes que viene es un descuento mas     no manana     no manana el otro el otro martes     es un descuento que le puedo dar del     casi  con  cancelan la deuda     perfecto pero lo puedo pagar el martes que viene     si usted le mando un link o algun codigo o algo para que yo pueda pagar     yo le voy a enviar tiene whatsapp en este numero     si     bueno yo le voy a enviar un whatsapp     el pago tiene que estar antes del mediodia si     a mas tardar a las  horas porque en ese horario hay que entregar los tickets al banco si     ahorita la voy a agregar al whatsapp y le voy a pasar el numero de cuenta donde tiene que pagar     es una cuenta corriente o un cbu     puede ir a cualquier santander con la cuenta corriente     o si no hace una transferencia con el

In [ ]:
prueba.pago[1]

['si senor como verifico que usted ya pago la deuda entonces estaria quedando con el',
 'entonces ahora ya verifico que dos meses no se pago entonces tuvo otra vez un incremento y todo esto']

In [ ]:
prueba.pago.astype(str).unique()

array(["['el pago tiene que estar antes del mediodia si']",
       "['si senor como verifico que usted ya pago la deuda entonces estaria quedando con el', 'entonces ahora ya verifico que dos meses no se pago entonces tuvo otra vez un incremento y todo esto']",
       "['que yo siempre tenia voluntad de pago']",
       "['ah bueno si quieren enviarlo de vuelta para tenerlo mas fresco pero apenas me depositen voy y les pago']",
       'nan', "['lo recuerdo y siempre pago un poquito de esto despues']",
       "['eso te queria decir porque ella pago', 'o sino que le vuelvo a escribir a rosa que es la persona que le hizo el acuerdo de pago', 'se cargo el pago']",
       "['para cuando podriamos agendar otra vez su pago', 'bueno esta bien le regiendo su pago entonces senor']",
       "['mientras yo espero el pago la acreditacion del pago de ayer si']",
       "['y si usted esta autorizada para cerrar un acuerdo de pago', 'le envie propuestas de pago', 'con un salpago y en cuota', 'si lo pone